In [1]:
#  Convolutional Neural Networks
#  is it Cat or Dog
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

import numpy as np
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
classifier = Sequential()
# number of filters   width, heights of each filter
# input shape forces all images to have same pixel size/format 64x64 size of image by pixels
# 3 : colored images (RGB)   1: black/white images
# relu activation turns all negative pixel values into zero
classifier.add(Convolution2D(32, 3, 3, input_shape=(64, 64, 3), activation='relu'))
# keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)
# pool_size =(x, y)   x : downsize vertical by this   y : downsize horizontal by this
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Convolution2D(32, 3, 3, activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# flattening layer
classifier.add(Flatten())
# fully connected layer
classifier.add(Dense(128, activation='relu'))
# final layer, since its only dog or cat (binary) we can use sigmoid instead of softmax
classifier.add(Dense(1, activation='sigmoid'))
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
callback_list = []
callback_list = [checkpoint]

C:\Users\Bob\AppData\Local\conda\conda\envs\Deep_learn\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., activation="relu")`
  
C:\Users\Bob\AppData\Local\conda\conda\envs\Deep_learn\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  # This is added back by InteractiveShellApp.init_path()


In [3]:
# image augmentation
# this function will add changes to our images so that we have more images (based on our dataset)
# this is useful when number of training elements in our dataset is too low
train_datagen = ImageDataGenerator(
    # here we rescale images, change the zoom and flip horizontal images so we have same image but
    # the model will view it as new data for training
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
# same as above but for test dataset
test_datagen = ImageDataGenerator(rescale=1./255)

# here we mention where to get the data from (location)
# size of our images
# batche size
# if it is two classes binary / more than two it is categorical
training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [4]:
# our data is structured this way:
#     dataset -> training_set -> cats -> images
#                             -> dogs -> images
#             -> test_set -> cats -> images
#                         -> dogs -> images
# this program will detect label of the image based on name of folder it is located in (cat or dog)
# so we wont need to feed it label of the images manually

In [5]:
# this is the method that will train our data and test it on validation (test) set
classifier.fit_generator(
        training_set,
        # how many training data do we have     
        steps_per_epoch=8000,
        epochs=5,
        validation_data=test_set,
        # how many testing data do we have     
        validation_steps=2000,callbacks=callback_list)

Epoch 1/5
8000/8000 [==============================] - 1128s 141ms/step - loss: 0.3611 - acc: 0.8315 - val_loss: 0.5843 - val_acc: 0.8044
Epoch 2/5
8000/8000 [==============================] - 1233s 154ms/step - loss: 0.1056 - acc: 0.9603 - val_loss: 0.9342 - val_acc: 0.8075
Epoch 3/5
8000/8000 [==============================] - 1164s 145ms/step - loss: 0.0514 - acc: 0.9815 - val_loss: 1.0517 - val_acc: 0.8009
Epoch 4/5
8000/8000 [==============================] - 1144s 143ms/step - loss: 0.0375 - acc: 0.9866 - val_loss: 1.1808 - val_acc: 0.7951
Epoch 5/5
8000/8000 [==============================] - 1135s 142ms/step - loss: 0.0306 - acc: 0.9897 - val_loss: 1.3091 - val_acc: 0.7950


In [6]:
# loading the image file and changing its size to 64x64 same as our training data
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size=(64,64))
# turning the image into 3d array with RGB layers
test_image = image.img_to_array(test_image)
# since our classifer recieves data in a batch we need to add 4th dimension (batch) to our single prediction data
test_image = np.expand_dims(test_image, axis=0)

result = classifier.predict(test_image)
print(training_set.class_indices)

{'cats': 0, 'dogs': 1}


In [7]:
print(result)

[[1.]]
